In [ ]:
# import sys
# 
# import bertopic
# import tqdm
# !{sys.executable} -m pip install stanza --upgrade
# !{sys.executable} -m pip install torch --upgrade
# 
# import matplotlib.pyplot as plt
# from scipy.cluster.hierarchy import dendrogram, linkage
# 
# def plot_dendrogram(embeddings, labels):
#     linked = linkage(embeddings, 'ward')
#     plt.figure(figsize=(12, 6))
#     dendrogram(linked, labels=labels, leaf_rotation=90)
#     plt.show()
# 
# import pandas as pd
# import ast
# import re
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
# from tqdm import tqdm
# 
# tqdm.pandas()  # Enables df.progress_apply
# 
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# nltk.download("stopwords")
# 
# stop_words = set(stopwords.words("english"))
# stemmer = PorterStemmer()
# 
# def normalize_text(text):
#     words = re.findall(r'[a-z0-9]+', text.lower())
#     return " ".join(stemmer.stem(w) for w in words if w not in stop_words)
# 
# 
# def extract_unique_amenities(df):
#     unique = set()
#     parsed_amenities = []
#     
#     for val in tqdm(df["amenities"], desc="Parsing amenities"):
#         try:
#             lst = ast.literal_eval(val) if isinstance(val, str) else []
#             lst = [normalize_text(str(a)) for a in lst]
#         except Exception:
#             lst = []
#         parsed_amenities.append(lst)
#         unique.update(lst)
#     print(len(unique))
#     
#     return list(unique), parsed_amenities
# 
# from collections import defaultdict
# 
# def cluster_amenities(unique_amenities, n_clusters=None):
#     print(f"Encoding {len(unique_amenities)} unique amenities...")
#     
# 
#     model = SentenceTransformer("all-MiniLM-L6-v2")
# 
#     embeddings = list(tqdm(model.encode(unique_amenities, show_progress_bar=True), 
#                             total=len(unique_amenities), 
#                             desc="Encoding embeddings"))
#     
#     if not n_clusters:
#         n_clusters = max(2, int(len(unique_amenities) ** 0.5))
#     
#     print(f"Clustering into {n_clusters} groups...")
#     clustering = AgglomerativeClustering(n_clusters=n_clusters)
#     labels = clustering.fit_predict(embeddings)
#     
#     # Group amenities by cluster
#     cluster_to_amenities = defaultdict(list)
#     for amenity, label in zip(unique_amenities, labels):
#         cluster_to_amenities[label].append(amenity)
#     
#     # Pick a representative name for each cluster
#     cluster_names = {}
#     for cluster, items in cluster_to_amenities.items():
#         # Pick the shortest name (after sorting)
#         rep_name = sorted(items, key=len)[0]
#         # Clean name for column use
#         rep_name = rep_name.replace(" ", "_").replace("-", "_")
#         cluster_names[cluster] = rep_name
#     print(cluster_names)
#     
#     # Map amenity → cluster name
#     amenity_to_cluster = {amenity: cluster_names[label] 
#                           for amenity, label in zip(unique_amenities, labels)}
#     plot_dendrogram(embeddings, unique_amenities)
# 
#     return amenity_to_cluster, cluster_to_amenities
# 
# 
# def expand_amenities_semantic(df, amenity_to_cluster, parsed_amenities):
#     cluster_names = set(amenity_to_cluster.values())
#     
#     # Initialize binary columns for each cluster name
#     for cname in cluster_names:
#         df[f"amenity_{cname}"] = 0
#     
#     for idx, lst in tqdm(enumerate(parsed_amenities), 
#                          total=len(parsed_amenities), 
#                          desc="Assigning amenities to clusters"):
#         cluster_ids = {amenity_to_cluster[a] for a in lst if a in amenity_to_cluster}
#         for cname in cluster_ids:
#             df.at[idx, f"amenity_{cname}"] = 1
#     
#     return df
# 
# def process_airbnb_with_semantic_amenities(path, n_clusters=None):
#     df = pd.read_csv(path)
#     unique_amenities, parsed_amenities = extract_unique_amenities(df)
#     print(type(unique_amenities))
#     amenity_to_cluster, cluster_to_amenities = cluster_amenities(unique_amenities, n_clusters)
#     df = expand_amenities_semantic(df, amenity_to_cluster, parsed_amenities)
#     return df, cluster_to_amenities
# 
# 
# if __name__ == "__main__":
#     df, mapping = process_airbnb_with_semantic_amenities(r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv")
#     # df.to_csv("cleaned_with_clusters.csv", index=False)
#     # print("Cluster mapping:", mapping)
# 
# df.head()


In [ ]:
# import pandas as pd
# import re
# import ast
# from sklearn.feature_extraction.text import CountVectorizer
# from tqdm import tqdm
# from collections import defaultdict
# import spacy
# 
# # --- NLP Setup ---
# try:
#     NLP = spacy.load("en_core_web_sm")
#     print("✓ spaCy NLP model loaded successfully.")
# except OSError:
#     print("Error: spaCy model 'en_core_web_sm' not found.")
#     print("Please run: python -m spacy download en_core_web_sm")
#     exit()
# 
# # --- Setup: Define Stop Words ---
# CUSTOM_STOP_WORDS = set(CountVectorizer(stop_words='english').get_stop_words()).union(['listing', 'available', 'extra cost', 'stay'])
# 
# tqdm.pandas(desc="Processing DataFrame")
# 
# def clean_amenity_text(text):
#     if not isinstance(text, str): return ""
# 
#     # 1. Convert to lowercase
#     text = text.lower()
# 
#     # --- NEW: Handle the "(word) allowed" pattern ---
#     # This regex finds a word followed by "allowed" and merges them.
#     # e.g., "pets allowed" -> "pets_allowed"
#     text = re.sub(r'(\w+)\s+allowed\b', r'\1_allowed', text)
# 
#     # 3. Remove other specific unwanted patterns
#     text = re.sub(r'\brequest\b', '', text)
#     text = re.sub(r'\d+\s+years?\s+old', '', text)
#     text = re.sub(r'ages?\s+\d+', '', text)
# 
#     # 4. Keep only lowercase English letters and underscores (for our new pattern)
#     text = re.sub(r'[^a-z\s_]', '', text)
#     
#     # 5. Remove stop words
#     words = text.split()
#     filtered_words = [word for word in words if word not in CUSTOM_STOP_WORDS]
#     text = ' '.join(filtered_words)
# 
#     # 6. Consolidate multiple spaces and strip
#     return re.sub(r'\s+', ' ', text).strip()
# 
# def _get_primary_object(phrase, nlp_model):
#     """
#     Uses a "right-most noun" heuristic to find the most important object.
#     """
#     doc = nlp_model(phrase)
#     try:
#         if not doc.noun_chunks:
#             return None
#         longest_chunk = max(doc.noun_chunks, key=lambda chunk: len(chunk.text.split()))
#         for token in reversed(longest_chunk):
#             if token.pos_ in ['NOUN', 'PROPN']:
#                 return token.lemma_
#         return longest_chunk.root.lemma_
#     except ValueError:
#         return None
# 
# def refine_and_group_amenities(candidates, nlp_model):
#     """
#     Applies the hybrid refinement process.
#     """
#     # --- Stage 1: Prune ---
#     print(f"--> Pruning {len(candidates)} initial candidates...")
#     candidates.sort(key=len, reverse=True)
#     temp_set = set(candidates)
#     for c in candidates:
#         if c in temp_set:
#             substrings = {other for other in temp_set if c != other and other in c}
#             temp_set.difference_update(substrings)
#     pruned_amenities = sorted(list(temp_set), key=len, reverse=True)
#     print(f"--> After pruning, {len(pruned_amenities)} candidates remain.")
# 
#     # --- Stage 2: Grouping with Hybrid Logic ---
#     ORDERED_SYNONYM_GROUPS = [
#         # == High-Priority Overrides ==
#         ('hair_dryer',        ['hair dryer']),
#         ('dishwasher',        ['dishwasher']),
#         ('washing_machine',   ['washer', 'washing machine']),
#         ('clothes_dryer',     ['dryer', 'tumble dryer']),
#         ('security_camera',   ['security camera', 'camera']),
#         ('pets_allowed',      ['pets_allowed']), # Catch our transformed pattern
#         ('smoking_allowed',   ['smoking_allowed']),
#         ('drying_rack',       ['drying rack']),
#         ('conditioner',       ['conditioner']), # Override for the NLP weakness
#         
#         # == General Concepts and Synonyms ==
#         ('air_conditioning',  ['air conditioning', 'ac', 'aircon']),
#         ('closet',            ['closet', 'wardrobe', 'dresser']),
#         ('tv',                ['tv', 'hdtv', 'television', 'hbo', 'cable', 'netflix', 'hulu', 'amazon prime', 'disney']),
#         ('wifi',              ['wifi', 'wireless internet', 'internet', 'ethernet']),
#         ('kitchen',           ['kitchen']),
#         ('coffee_maker',      ['coffee', 'coffee maker', 'nespresso', 'keurig']),
#         ('dinnerware',        ['dinnerware']),
#         ('heating',           ['heating', 'heater']),
#         ('parking',           ['parking']),
#         ('pool',              ['pool']),
#         ('hot_tub',           ['hot tub', 'jacuzzi']),
#         ('bathtub',           ['bath', 'bathtub']),
#         ('patio_balcony',     ['patio', 'balcony']),
#         ('gym',               ['gym', 'fitness']),
#         ('first_aid_kit',     ['first aid']),
#         ('smoke_alarm',       ['smoke alarm', 'smoke detector']),
#         ('carbon_monoxide_alarm', ['carbon monoxide alarm', 'co alarm', 'carbon monoxide detector']),
#         ('fire_extinguisher', ['fire extinguisher']),
#         ('refrigerator',      ['refrigerator', 'fridge']),
#         ('microwave',         ['microwave']),
#         ('oven',              ['oven']),
#         ('stove',             ['stove', 'cooktop']),
#         ('books',             ['books']),
#         ('waterfront',        ['river', 'canal', 'waterfront']),
#         ('shampoo',           ['shampoo']),
#         ('body_soap',         ['soap', 'body soap'])
#     ]
# 
#     groups = defaultdict(list)
#     
#     # First pass: Use the high-precision synonym list
#     amenities_to_process = set(pruned_amenities)
#     for standard_name, synonyms in tqdm(ORDERED_SYNONYM_GROUPS, desc="Grouping by Synonyms"):
#         matched_this_pass = set()
#         for amenity in amenities_to_process:
#             for synonym in synonyms:
#                 if re.search(r'\b' + re.escape(synonym) + r'\b', amenity):
#                     groups[standard_name].append(amenity)
#                     matched_this_pass.add(amenity)
#                     break 
#         amenities_to_process.difference_update(matched_this_pass)
# 
#     # Second pass: Use NLP as a fallback for everything else
#     print(f"--> {len(amenities_to_process)} amenities remaining for NLP fallback grouping...")
#     for amenity in tqdm(list(amenities_to_process), desc="Grouping by NLP"):
#         primary_object = _get_primary_object(amenity, nlp_model)
#         if primary_object:
#             groups[primary_object].append(amenity)
#         else:
#             groups[amenity].append(amenity)
# 
#     print(f"--> Consolidated into {len(groups)} final amenity groups.")
#     return groups
# 
# def standardize_amenities_final(file_path, min_df_threshold=50):
#     try:
#         nlp = spacy.load("en_core_web_sm")
#     except OSError:
#         return None
# 
#     df = pd.read_csv(file_path, engine='python', on_bad_lines='warn')
# 
#     df['parsed_amenities'] = df['amenities'].progress_apply(
#         lambda s: ast.literal_eval(s.strip()) if isinstance(s, str) and s.strip().startswith('[') else []
#     )
#     
#     corpus = [clean_amenity_text(a) for l in df['parsed_amenities'] for a in l if clean_amenity_text(a)]
#     
#     vectorizer = CountVectorizer(ngram_range=(1, 5), min_df=min_df_threshold)
#     vectorizer.fit(corpus)
#     initial_candidates = vectorizer.get_feature_names_out().tolist()
# 
#     final_amenity_groups = refine_and_group_amenities(initial_candidates, nlp)
#     print(final_amenity_groups)
#     df['cleaned_amenities_text'] = df['parsed_amenities'].progress_apply(
#         lambda lst: ' | '.join([clean_amenity_text(item) for item in lst])
#     )
# 
#     for group_name, search_terms in tqdm(final_amenity_groups.items(), desc="Creating Columns"):
#         column_name = f"has_{group_name.replace(' ', '_')}"
#         pattern = '|'.join([r'\b' + re.escape(term) + r'\b' for term in search_terms])
#         df[column_name] = df['cleaned_amenities_text'].str.contains(pattern, na=False).astype(int)
# 
#     df = df.drop(columns=['parsed_amenities', 'cleaned_amenities_text'])
#     print("✓ Binary columns created successfully.")
#     return df
# 
# # --- Main execution block ---
# if __name__ == '__main__':
#     csv_file_path = r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv"
#     MIN_DF_THRESHOLD = 100 
# 
#     transformed_df = standardize_amenities_final(
#         csv_file_path, min_df_threshold=MIN_DF_THRESHOLD
#     )
# 
#     if transformed_df is not None:
#         print("\nTransformation complete. Here's a preview:")
#         amenity_cols = sorted([col for col in transformed_df.columns if col.startswith('has_')])
#         display_cols = ['id', 'name'] + amenity_cols
#         
#         if len(display_cols) > 20:
#             print(f"(Showing a subset of the {len(amenity_cols)} new amenity columns)")
#             display_cols = display_cols[:20]
# 
#         print(transformed_df[display_cols].head())

In [ ]:
# csv_file_path = r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv"
# MIN_DF_THRESHOLD = 100 
# 
# 
# transformed_df = standardize_amenities_final(
#     csv_file_path, min_df_threshold=MIN_DF_THRESHOLD
# )
# 
# if transformed_df is not None:
#     print("\nTransformation complete. Here's a preview:")
#     amenity_cols = sorted([col for col in transformed_df.columns if col.startswith('has_')])
#     display_cols = ['id', 'name'] + amenity_cols
#     
#     if len(display_cols) > 20:
#         print(f"(Showing a subset of the {len(amenity_cols)} new amenity columns)")
#         display_cols = display_cols[:20]
# 
#     print(transformed_df[display_cols].head())

In [12]:
import pandas as pd
import re
import ast
from tqdm import tqdm
from collections import defaultdict
import spacy
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
tqdm.pandas(desc="Processing Amenities")

# --- NLP Setup ---
try:
    NLP = spacy.load("en_core_web_md")
    print("✓ spaCy NLP model (md) with word vectors loaded successfully.")
except OSError:
    print("Error: spaCy model 'en_core_web_md' not found.")
    print("Please run: python -m spacy download en_core_web_md")
    exit()

# --- Setup: Define Stop Words ---
# Combine standard English stop words with your custom list for efficiency.
CUSTOM_STOP_WORDS = set(CountVectorizer(stop_words='english').get_stop_words()).union(['listing', 'extra', 'cost', 'included', 'private'])

# --- Cleaning Function ---
def clean_amenity_text(text):
    """
    Cleans a raw amenity string for better classification.
    """
    if not isinstance(text, str):
        return ""

    # 1. Convert to lowercase
    text = text.lower()
    
    # 2. Keep only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Remove stop words and custom words
    words = text.split()
    filtered_words = [word for word in words if word not in CUSTOM_STOP_WORDS]
    text = ' '.join(filtered_words)
    
    # 4. Consolidate whitespace
    return re.sub(r'\s+', ' ', text).strip()

# --- Semantic Classifier Class ---
class AmenityClassifier:
    def __init__(self, category_definitions):
        self.category_vectors = {}
        print("Calculating semantic vectors for categories...")
        for category, examples in tqdm(category_definitions.items(), desc="Vectorizing Categories"):
            example_vectors = [NLP(example).vector for example in examples if NLP(example).has_vector]
            if example_vectors:
                self.category_vectors[category] = np.mean(example_vectors, axis=0)

    def _cosine_similarity(self, vec1, vec2):
        if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
            return 0
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def predict(self, amenity_text, threshold=0.4):
        # The text passed to predict should now be pre-cleaned
        if not amenity_text or not NLP(amenity_text).has_vector:
            return None
        amenity_vector = NLP(amenity_text).vector
        best_category, max_similarity = None, -1
        for category, category_vector in self.category_vectors.items():
            similarity = self._cosine_similarity(amenity_vector, category_vector)
            if similarity > max_similarity:
                max_similarity = similarity
                best_category = category
        if max_similarity > threshold:
            return best_category
        return None

# --- Main Logic ---
AMENITY_CATEGORIES = {
    'safety':         ['smoke alarm', 'carbon monoxide alarm', 'fire extinguisher', 'first aid'],
    'kitchen':        ['kitchen', 'refrigerator', 'fridge', 'microwave', 'oven', 'stove', 'cooktop', 'dishwasher', 'kettle', 'hot water kettle', 'blender', 'baking'],
    'laundry':        ['washer', 'washing machine', 'dryer', 'tumble dryer'],
    'air_conditioner':['air conditioning', 'ac', 'aircon', 'fan'],
    'heater':         ['heating', 'heater', 'fireplace'],
    'hygiene':        ['shampoo', 'conditioner', 'soap', 'body soap', 'hot tub', 'jacuzzi', 'bathtub', 'bath', 'shower', 'hot water'],
    'bedroom_storage':['closet', 'wardrobe', 'dresser', 'hangers', 'drying rack'],
    'entertainment':  ['tv', 'hdtv', 'television', 'hbo', 'cable', 'netflix', 'hulu', 'amazon prime', 'disney', 'books', 'games', 'console'],
    'internet':       ['wifi', 'wireless internet', 'internet', 'ethernet'],
    'parking':        ['parking'],
    'exercise':       ['gym', 'fitness', 'climbing wall'],
    'outdoors':       ['patio', 'balcony', 'grill', 'bbq'],
    'children_care':  ['crib', 'high chair', 'childrens dinnerware', 'children', 'baby'],
    'special_access': ['pets allowed', 'smoking allowed'],
    'coffee_maker':   ['coffee', 'coffee maker', 'nespresso', 'keurig'],
    'cooking_basics': ['cooking basics', 'dinnerware'],
    'sound_system': ['sound', 'speakers', 'sound system'],
    'accessibility': ['elevator', 'lift', 'ramp', 'ground floor'],
    'view': ['view']
}

CATEGORY_REPRESENTATIVES = {
    'exercise':       ['gym', 'exercise equipment', 'weights', 'treadmill', 'pool', 'yoga mat', 'ping pong table'],
    'kitchen':        ['kitchen', 'appliance', 'refrigerator', 'oven', 'stove', 'fridge'],
    'safety':         ['safety', 'security', 'alarm', 'detector', 'lock', 'first aid'],
    'entertainment':  ['entertainment', 'tv', 'sound system', 'game console', 'books', 'board games'],
    'outdoors':       ['outdoors', 'backyard', 'garden', 'patio', 'balcony', 'grill', 'bbq'],
    'cooking_basics': ['cooking basics', 'dinnerware', 'utensils', 'cups', 'glasses', 'forks', 'spoons', 'pots']
}

def classify_single_amenity(raw_amenity, category_map, classifier):
    """
    Classifies a single raw amenity string using the hybrid approach.
    Now uses a pre-cleaning step.
    """
    raw_amenity_lower = raw_amenity.lower()

    # Pass 1: High-Precision Dictionary Lookup on the raw (but lowercased) text
    for category, search_terms in category_map.items():
        for term in search_terms:
            if re.search(r'\b' + re.escape(term) + r'\b', raw_amenity_lower):
                return category

    # Pass 2: Semantic Fallback on the fully cleaned text
    cleaned_amenity = clean_amenity_text(raw_amenity)
    return classifier.predict(cleaned_amenity)

def inspect_amenity_classification(file_path):
    """
    Loads data, classifies all unique amenities, and prints the results
    for review without modifying the DataFrame.
    """
    classifier = AmenityClassifier(CATEGORY_REPRESENTATIVES)
    df = pd.read_csv(file_path, engine='python', on_bad_lines='warn')

    df['parsed_amenities'] = df['amenities'].progress_apply(
        lambda s: ast.literal_eval(s.strip()) if isinstance(s, str) and s.strip().startswith('[') else []
    )
    
    all_unique_amenities = set()
    for amenity_list in df['parsed_amenities']:
        all_unique_amenities.update(amenity_list)

    print(f"\nFound {len(all_unique_amenities)} unique amenities to classify.")
    
    final_groups = defaultdict(list)

    for raw_amenity in tqdm(sorted(list(all_unique_amenities)), desc="Classifying Amenities"):
        category = classify_single_amenity(raw_amenity, AMENITY_CATEGORIES, classifier)
        if category:
            final_groups[category].append(raw_amenity)
        else:
            final_groups['__UNCLASSIFIED__'].append(raw_amenity)

    # --- FINAL DIAGNOSTIC OUTPUT ---
    print("\n--- Amenity Classification Review ---")
    for category, amenities in sorted(final_groups.items()):
        print(f"\n--- Category: {category} ({len(amenities)} items) ---")
        # Print a sample of up to 15 amenities for brevity
        sample_amenities = amenities
        for amenity in amenities[:100]:
            print(f"  - {amenity}")
        if len(amenities) > 100:
            print(f"  - ... and {len(amenities) - 100} more.")

# --- Main execution block ---
if __name__ == '__main__':
    csv_file_path = r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv"
    final_groups = inspect_amenity_classification(csv_file_path)

✓ spaCy NLP model (md) with word vectors loaded successfully.
Calculating semantic vectors for categories...


Processing Amenities: 100%|██████████| 10168/10168 [00:00<00:00, 13950.02it/s]



Found 3067 unique amenities to classify.


Classifying Amenities: 100%|██████████| 3067/3067 [00:02<00:00, 1064.23it/s]


--- Amenity Classification Review ---

--- Category: __UNCLASSIFIED__ (37 items) ---
  - Babysitter recommendations
  - Bidet
  - Bikes
  - Boat slip
  - Building staff
  - Canal view
  - Courtyard view
  - EV charger
  - Elevator
  - Essentials
  - Fire pit
  - Free carport on premises
  - Harbor view
  - Host greets you
  - Housekeeping - available at extra cost
  - Indoor fireplace
  - Indoor fireplace: wood-burning
  - Iron
  - Kayak
  - Keypad
  - Kitchenette
  - Lake access
  - Lake view
  - Lockbox
  - Marina view
  - Ocean view
  - Piano
  - Private sauna
  - Record player
  - Room-darkening shades
  - Sauna
  - Sea view
  - Self check-in
  - Ski-in/Ski-out
  - Sun loungers
  - Toaster
  - Waterfront

--- Category: air_conditioner (6 items) ---
  - AC - split type ductless system
  - Air conditioning
  - Ceiling fan
  - Central air conditioning
  - Portable air conditioning
  - Window AC unit

--- Category: bedroom_storage (17 items) ---
  - Clothing storage: closet
  - Clothi

In [9]:
final_groups

defaultdict(list,
            {'kitchen': [' Boretti  stainless steel gas stove',
              ' Boretti stainless steel gas stove',
              ' Boretti stainless steel oven',
              "* Use 'Liebherr' fridge in kitchen. * I wash the dishes. refrigerator",
              '. refrigerator',
              '100Lt refrigerator',
              '2 fridges  refrigerator',
              '2 minibars refrigerator',
              '2 ovens oven',
              '? double oven',
              'A-Brand refrigerator',
              'A-Brand stainless steel gas stove',
              'A-Brand stainless steel oven',
              'AEG  oven',
              'AEG  refrigerator',
              'AEG  stainless steel single oven',
              'AEG Micromat Combi oven',
              'AEG double oven',
              'AEG electric stove',
              'AEG gas stove',
              'AEG induction stove',
              'AEG oven',
              'AEG refrigerator',
              'AEG single oven',
   